In [ ]:
import pickle
import open3d as o3d
import numpy as np
from tqdm.auto import tqdm
%matplotlib inline

%load_ext autoreload
%autoreload 2
from digiforest_analysis.tasks.tree_reconstruction import Tree, Circle
from digiforest_analysis_ros.forest_analysis import TreeManager

In [ ]:
def text_3d(text, pos, direction=None, degree=0.0, font='DejaVuSansMono.ttf', font_size=16):
    """
    Generate a 3D text point cloud used for visualization.
    :param text: content of the text
    :param pos: 3D xyz position of the text upper left corner
    :param direction: 3D normalized direction of where the text faces
    :param degree: in plane rotation of text
    :param font: Name of the font - change it according to your system
    :param font_size: size of the font
    :return: o3d.geoemtry.PointCloud object
    """
    if direction is None:
        direction = (0., 0., 1.)

    from PIL import Image, ImageFont, ImageDraw
    from pyquaternion import Quaternion

    font_obj = ImageFont.truetype(font, font_size)
    font_dim = font_obj.getbbox(text)[2:]

    img = Image.new('RGB', font_dim, color=(255, 255, 255))
    draw = ImageDraw.Draw(img)
    draw.text((0, 0), text, font=font_obj, fill=(0, 0, 0))
    img = np.asarray(img)
    img_mask = img[:, :, 0] < 128
    indices = np.indices([*img.shape[0:2], 1])[:, img_mask, 0].reshape(3, -1).T

    pcd = o3d.geometry.PointCloud()
    pcd.colors = o3d.utility.Vector3dVector(img[img_mask, :].astype(float) / 255.0)
    pcd.points = o3d.utility.Vector3dVector(indices / 100.0)

    raxis = np.cross([0.0, 0.0, 1.0], direction)
    if np.linalg.norm(raxis) < 1e-6:
        raxis = (0.0, 0.0, 1.0)
    trans = (Quaternion(axis=raxis, radians=np.arccos(direction[2])) *
             Quaternion(axis=direction, degrees=degree)).transformation_matrix
    trans[0:3, 3] = np.asarray(pos)
    pcd.transform(trans)
    return pcd

In [ ]:
manager_path = "/home/ori/git/digiforest_drs/digiforest_analysis_ros/output/trees/logs/raw/tree_manager_aggregate.pkl"

with open(manager_path, "rb") as f:
    tm : TreeManager = pickle.load(f) 

tree_id = None
if tree_id is not None:
    index = [t.id for t in tm.trees].index(tree_id)
    tree_iter = tqdm(tm.trees[index:index+1])
else:
    tree_iter = tqdm(tm.trees)


viz_objs = []
for tree in tree_iter:
    tree.reconstruct2(max_center_deviation=0.5, force_straight=True)

    verts, tris = tree.generate_mesh()
    mesh = o3d.geometry.TriangleMesh(
        o3d.utility.Vector3dVector(verts),
        o3d.utility.Vector3iVector(tris)
    )
    mesh.compute_vertex_normals()
    
    label_pos = tree.axis["transform"][:3, 3]
    label_pos[2] = -3
    label = text_3d(f"Tree {tree.id:0>3}", label_pos, direction=(0, 0, 1), font_size=30)
    
    viz_objs.append(mesh)
    viz_objs.append(tree.clusters[0]["cloud"].to_legacy())
    viz_objs.append(label)
o3d.visualization.draw_geometries(viz_objs)